In [31]:
# !pip install voila
# !jupyter serverextension enable voila --sys-prefix


In [221]:
from fastai.vision.all import *
from fastai.vision.widgets import *
import pickle as pkl
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from IPython.display import display, FileLink


In [222]:
btn_upload = widgets.FileUpload()
out = widgets.Output(layout={'border': '1px solid black'})
lbl_widget = widgets.Label('Please upload your CSV for prediction')


In [223]:
def on_click(change):
    with open('genetic_v1.pickle', 'rb') as f:
        model = pkl.load(f)   
    with open("input.csv", "w+b") as i:
        i.write(btn_upload.data[0])
    test_df = pd.read_csv("input.csv",index_col=0).rename(columns={"0.5*smart_12_raw - 0.99*smart_242_raw": "eng1", "-1.04*smart_192_raw + 0.67*smart_241_raw": "eng2"})
    X = test_df.drop(columns=['target'])
    y_test = test_df['target']
    y_pred = model.predict(X)
    probas_pred = model.predict_proba(X)
    result = pd.read_csv("input.csv",index_col=0)
    result['preds'] = y_pred
    result.to_csv('output.csv')
    with out:
        out.clear_output()
        print('\n -----These are the scores of the model:----- \n')
        print("recall: ", recall_score(y_test, y_pred))
        print("precision: ",precision_score(y_test, y_pred))
        print("f1: ",f1_score(y_test,y_pred))
        print("\nconfusion matrix: \n",confusion_matrix(y_test,y_pred))
        local_file = FileLink('./output.csv', result_html_prefix="Click here to download: ")
        display(local_file)


In [224]:
btn_upload.observe(on_click, names=['data'])

In [225]:
display(VBox([lbl_widget,btn_upload,out]))